In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('text', usetex=True)
matplotlib.rcParams['text.latex.preamble']=r"\usepackage{amsmath}"


from stuff import *
from utils import *
float_formatter = lambda x: "%.2e" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
plt.rcParams.update({'font.size': 14})
%config InlineBackend.figure_format = 'retina'

from time import time
import cProfile
import pstats



In [ ]:
iters = 1000
np.random.seed(42)
plt.figure(figsize=(12,5))

model_params = dict(nodes=10, dim=100, B_rank=1, graph="erdos-renyi", edge_prob=0.2)
model = Model(**model_params)

mu_x, mu_xy, L_x, L_xy = model.get_mu_L()

t_start = time()
x_res, f_err, cons_err = APDM(iters, model)
print(f"\ntime APDM: {time() - t_start}\n")

plt.subplot(121)
plt.plot(f_err, label='APDM')
plt.title('$|F(\\boldsymbol{x}^k) -F^*|$')
plt.yscale('log')

plt.subplot(122)
plt.plot(cons_err, label='APDM')
plt.title('$||\\boldsymbol{Ax}^k||$')
plt.yscale('log')

C = 2.5  # big O const
N = np.arange(iters)
plt.subplot(121)
first_guy = (L_x / mu_x)**0.5 * L_xy / mu_xy
second_guy = (L_xy / mu_xy) ** 2
rate_const = max(first_guy, second_guy)
print("first_guy", first_guy, "second_guy", second_guy, "rate_const", rate_const)

# plt.plot(f_err[0] * np.exp(-N / (C * rate_const)), label='APDM upper')



# x_res, f_err, cons_err = LDM(iters, model)

# plt.subplot(121)
# plt.plot(f_err, label='locally dual nonacc')
# plt.legend()

# plt.subplot(122)
# plt.plot(cons_err, label='locally dual nonacc')
# plt.legend()


# x_res, f_err, cons_err = GDM(iters, model)

# plt.subplot(121)
# plt.plot(f_err, label='globally dual nonacc')
# plt.legend()

# plt.subplot(122)
# plt.plot(cons_err, label='globally dual nonacc')
# plt.legend()

t_start = time()
x_res, f_err, cons_err = GDAM(iters, model)
print(f"\ntime GDAM: {time() - t_start}\n")

plt.subplot(121)
plt.plot(f_err, label='Globally Dual')
plt.legend()

plt.subplot(122)
plt.plot(cons_err, label='Globally Dual')
plt.legend()

t_start = time()
x_res, f_err, cons_err = LDAM(iters, model)
print(f"\ntime LDAM: {time() - t_start}\n")

plt.subplot(121)
plt.plot(f_err, label='Locally Dual')
plt.legend()
plt.xlabel('Iteration number')

plt.subplot(122)
plt.plot(cons_err, label='Locally Dual')
plt.legend()
plt.xlabel('Iteration number')

plt.savefig("n10_d100.eps")

In [ ]:
%%time
iters=4000
repeat = 10 
t_apdm, t_gdam, t_ldam = 0, 0, 0
it_apdm, it_gdam, it_ldam = 0, 0, 0
for _ in range(repeat):
    model = Model(**model_params)
    t_start = time()
    x_res, f_err, cons_err = APDM(iters, model, use_crit=True)
    t_apdm += time() - t_start
    it_apdm += (f_err > 0).sum()  # error arrays initialized as np.zeros()
    
    t_start = time()
    x_res, f_err, cons_err = GDAM(iters, model, use_crit=True)
    t_gdam += time() - t_start
    it_gdam += (f_err > 0).sum() 
    
    
    t_start = time()
    x_res, f_err, cons_err = LDAM(iters, model, use_crit=True)
    t_ldam += time() - t_start
    it_ldam += (f_err > 0).sum() 
    
print(f"\ntime APDM (crit): {t_apdm / repeat}, iters: {it_apdm / repeat}\n")  
print(f"\ntime GDAM (crit): {t_gdam / repeat}, iters: {it_gdam / repeat}\n")
print(f"\ntime LDAM (crit): {t_ldam / repeat}, iters: {it_ldam / repeat}\n")

In [ ]:
profiler = cProfile.Profile()
profiler.enable()
APDM(iters, model)
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()


In [ ]:
profiler = cProfile.Profile()
profiler.enable()
GDAM(iters, model)
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()


In [ ]:
iters = 1000
plt.figure(figsize=(12,5))

model = Model(nodes=4, dim=15)


params = list(get_apdm_params(model))
x_res, f_err, cons_err = APDM(iters, model, params)


plt.subplot(121)
plt.plot(f_err, label=f'APDM, auto')
plt.title('$|f -f^*|$')
plt.yscale('log')

plt.subplot(122)
plt.plot(cons_err, label=f'APDM, auto')
plt.title('$||Ax||$')
plt.yscale('log')


params = list(get_apdm_params(model))

x_params, y_params = params
eta_x, alpha_x, beta_x, tau_x, sigma_x = x_params
theta, eta_y, alpha_y, beta_y, tau_y, sigma_y = y_params

# eta_x *= 2 
# eta_y *= 2 
# theta=0.999
# sigma_x = 0.9
# sigma_y = 0.5
beta_x = 0.0001
beta_y = 0.1

x_params = eta_x, alpha_x, beta_x, tau_x, sigma_x
y_params = theta, eta_y, alpha_y, beta_y, tau_y, sigma_y

x_res, f_err, cons_err = APDM(iters, model, (x_params, y_params))

plt.subplot(121)
plt.plot(f_err, label=f'APDM, manual params')
plt.legend()

plt.subplot(122)
plt.plot(cons_err, label=f'APDM, manual params')
plt.legend()

In [ ]:
# M = model.A.T @ model.A
from numpy import polynomial as P
# np.random.seed(42)
M = np.random.random((3,3))
M = M.T @ M

chi = lambda M: utils.lambda_max(M) / utils.lambda_min_plus(M)
gamma = 1 / chi(M)
K = int(1 / gamma**0.5 )

print("chi", 1/gamma, "pol_degree", K)
Tk = P.Chebyshev.basis(K)
Tk = Tk.convert(kind=P.Polynomial)  # move to power basis

# print("Tk coefs", Tk.coef)
c2 = (1 + gamma) / (1 - gamma) 
Pk = 1 - Tk(P.Polynomial([c2, -c2])) / Tk(c2)

# Pk_coefs = np.polynomial.chebyshev.cheb2poly(Pk.coef)
# print("Pk coefs", Pk.coef)

# powers = np.array([np.linalg.matrix_power(M, i) for i in range(len(Pk.coef))])
powers = [np.eye(M.shape[0])]
for i in range(1, len(Pk.coef)):
    powers.append(powers[-1] @ M)
    
M_ = (powers * Pk.coef[:, np.newaxis, np.newaxis]).sum(axis=0)

print(chi(M_))




In [ ]:
lmax, lminp = utils.lambda_max(M), utils.lambda_min_plus(M)
print(lmax, lminp )
diff = lmax - lminp
lams = np.linspace(lminp - diff / 100, lmax + diff / 100, 100)
plt.scatter(lminp, Pk(lminp))
plt.scatter(lmax, Pk(lmax))
print(Pk(lmax) / Pk(lminp))
# xx, yy = Pk.linspace()
vals = [Pk(lam) for lam in lams]
plt.plot(lams, vals)